# Exploring conformational space of selected macrocycles - "M1"; <br /> Part 1: Generation and selection of conformer candidates (MM methods)

In [50]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [63]:
import glob
import py3Dmol

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline 

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Mon May 29 08:36:26 2017


In [157]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def write_to_dict(prefix, suppl):
    moldict = {}
    for i, mol in enumerate(suppl):
        name = prefix + str(i)
        moldict[name] = mol
    return moldict

def align_structures_to_crystal(moldict):
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))

def align_structures_to_lowest_energy(moldict, energy_dict):
    """
    align structures from the structures' dictionary
    
    note that the dictionary in python is not ordered, so the "first" element is arbitrary
    """
    energy_sorted = sorted(energy_dict.items(), key=lambda x: x[1])
    first = energy_sorted[0][0]
    print("first = ", first)
    print("moldict = ", moldict)
    print("energy_dict = ", energy_dict)
    core_first = moldict[first].GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,moldict[first],atomMap=list(zip(core_mol,core_first)))        

def prepare_view(moldict):
    p = py3Dmol.view(width=400,height=400)
    for key, mol in moldict.items():
        mb = Chem.MolToMolBlock(mol)
        p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.15'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p        

def make_similarity_matrix(moldict):
    
    similarity_matrix = {}

    for k1, m1 in moldict.items():
        for k2, m2 in moldict.items():
            if (k1, k2) in similarity_matrix.keys() or (k2, k1) in similarity_matrix.keys():
                pass
            else:
                if k1 != k2:
                    rms = AllChem.GetBestRMS(Chem.RemoveHs(m1),Chem.RemoveHs(m2))
                    similarity_matrix[(k1, k2)] = rms 
                    
    return similarity_matrix


def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

## Crystal structure of "M1" macrocycle

In [65]:
cm1 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.xyz','r').read()
vcm1 = py3Dmol.view(width=400,height=400)
vcm1.removeAllModels()
vcm1.addModel(cm1,'xyz')
vcm1.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm1.setBackgroundColor('0xeeeeee')
vcm1.zoomTo()
vcm1.show()

In [66]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'
core_smiles = 'n1ccccc1'

m1 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.sdf')
m1_crystal = templ_m1[0]

## Conformers generated with the Balloon software:

Conformers were generated using the genetic algorithm as implemented in the Balloon software:

* starting with the crystal geometry kept as a template, results with prefix: "m1_b_sdf"; the crystal is of the "ss-ss" type;

* starting with the SMILES signature of M1 and allowing to "rebuild the geometry" (option --rebuildGeometry), results with prefix: "m1_b_smi"

* starting with structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

    where "ss\_sa" means "(syn-syn)\_(syn-anti)" configuration, etc. with the bracket notation used to mark conformations around the rings.
    

In all cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

In [106]:
inps_m1_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsdf/*.sdf')
inps_m1_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsmiles/*.sdf')
inps_m1_b_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_ss_sa/*.sdf')
inps_m1_b_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_ss_aa/*.sdf')
inps_m1_b_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_sa/*.sdf')
inps_m1_b_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_as/*.sdf')
inps_m1_b_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_aa/*.sdf')
inps_m1_b_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_aa_aa/*.sdf')

In [107]:
e_m1_b_sdf = grep_energies_from_sdf_outputs(inps_m1_b_sdf)
e_m1_b_smi = grep_energies_from_sdf_outputs(inps_m1_b_smi)
e_m1_b_ss_sa = grep_energies_from_sdf_outputs(inps_m1_b_ss_sa)
e_m1_b_ss_aa = grep_energies_from_sdf_outputs(inps_m1_b_ss_aa)
e_m1_b_sa_sa = grep_energies_from_sdf_outputs(inps_m1_b_sa_sa)
e_m1_b_sa_as = grep_energies_from_sdf_outputs(inps_m1_b_sa_as)
e_m1_b_sa_aa = grep_energies_from_sdf_outputs(inps_m1_b_sa_aa)
e_m1_b_aa_aa = grep_energies_from_sdf_outputs(inps_m1_b_aa_aa)

In [108]:
# write conformers to dictionaries
   
suppl_m1_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_sdfout.sdf')
suppl_m1_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_smilesout.sdf')
suppl_m1_b_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_ss_sa_sdfout.sdf')
suppl_m1_b_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_ss_aa_sdfout.sdf')
suppl_m1_b_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_sa_sdfout.sdf')
suppl_m1_b_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_as_sdfout.sdf')
suppl_m1_b_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_aa_sdfout.sdf')
suppl_m1_b_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_aa_aa_sdfout.sdf')

allmol_m1_b_sdf   = write_to_dict("m1_b_sdf_", suppl_m1_b_sdf)
allmol_m1_b_smi   = write_to_dict("m1_b_smi_", suppl_m1_b_smi)
allmol_m1_b_ss_sa = write_to_dict("m1_b_ss_sa_", suppl_m1_b_ss_sa)
allmol_m1_b_ss_aa = write_to_dict("m1_b_ss_aa_", suppl_m1_b_ss_aa)
allmol_m1_b_sa_sa = write_to_dict("m1_b_sa_sa_", suppl_m1_b_sa_sa)
allmol_m1_b_sa_as = write_to_dict("m1_b_sa_as_", suppl_m1_b_sa_as)
allmol_m1_b_sa_aa = write_to_dict("m1_b_sa_aa_", suppl_m1_b_sa_aa)
allmol_m1_b_aa_aa = write_to_dict("m1_b_aa_aa_", suppl_m1_b_aa_aa)

The generated conformer structures are presented in a separate notebook: [link]()

### pre-screening

All the generated structures are pre-optimized with MM methods (MMFF94-like force field). To remove potential duplicates, we will:

* calculate the root-mean-square-distance (RMSD) between the pairs of conformers (taking into account the heavy atoms only);
* compare the energies of conformers

In [143]:
allmol_m1_b = {}
allmol_m1_b.update(allmol_m1_b_sdf)
allmol_m1_b.update(allmol_m1_b_smi)
allmol_m1_b.update(allmol_m1_b_ss_sa)
allmol_m1_b.update(allmol_m1_b_ss_aa)
allmol_m1_b.update(allmol_m1_b_sa_sa)
allmol_m1_b.update(allmol_m1_b_sa_as)
allmol_m1_b.update(allmol_m1_b_sa_aa)
allmol_m1_b.update(allmol_m1_b_aa_aa)

print("The total number of generated conformers = ", len(allmol_m1_b))
             
energy_m1_b = {}
energy_m1_b.update(e_m1_b_sdf)
energy_m1_b.update(e_m1_b_smi)
energy_m1_b.update(e_m1_b_ss_sa)
energy_m1_b.update(e_m1_b_ss_aa)
energy_m1_b.update(e_m1_b_sa_sa)
energy_m1_b.update(e_m1_b_sa_as)
energy_m1_b.update(e_m1_b_sa_aa)
energy_m1_b.update(e_m1_b_aa_aa)

The total number of generated conformers =  94


In [144]:
# 1. calculate the similarity matrix between all pairs of conformers and sort its elements from the lowest
# (the most similar structures) to the largest values (the most different structures)
similarity_matrix_b = make_similarity_matrix(allmol_m1_b)
similarity_matrix_b_sorted = sorted(similarity_matrix_b.items(), key=lambda x: x[1])

# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than threshold ("similarity_thresh"), 
# compare energies; then if the energies are similar (controlled by the "energy_thresh"), 
#then remove the one with higher energy

to_be_deleted = []
similarity_thresh = 0.5 # Angstrom (same as Balloon's original RMSD threshold used for conformer generation)
energy_thresh     = 1 # kcal/mol (Balloon original threshold used for conformer generation is 25 kcal/mol)

for pair in similarity_matrix_b_sorted:
    if pair[1] < similarity_thresh:
        conf1 = pair[0][0]
        conf2 = pair[0][1]
        if abs(energy_m1_b[conf1] - energy_m1_b[conf2]) < energy_thresh:
            #print("conf1, conf2, E(conf1), E(conf2) = ", conf1, conf2, energy_m1_b[conf1], energy_m1_b[conf2])
            if energy_m1_b[conf1] < energy_m1_b[conf2]:
                to_be_deleted.append(conf2)
            else:
                to_be_deleted.append(conf1)

for mol in to_be_deleted:
    print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix_b.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix_b[k]
    allmol_m1_b.pop(mol, None)
    energy_m1_b.pop(mol, None)


conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_5 m1_b_ss_sa_8 60.09147762992992 60.092471088198806
conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_2 m1_b_ss_sa_4 57.28858557930001 57.07186921216874
conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_0 m1_b_sa_sa_0 53.54579276470567 53.59591327407036
conf1, conf2, E(conf1), E(conf2) =  m1_b_smi_3 m1_b_ss_sa_2 54.7449085712738 55.27137672504411
conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_1 m1_b_ss_sa_2 55.58881996735126 55.27137672504411
conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_10 m1_b_smi_7 63.51738573458692 62.82731031289344
conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_6 m1_b_ss_sa_7 60.171930667750985 59.669144350630006
conf1, conf2, E(conf1), E(conf2) =  m1_b_sa_as_1 m1_b_smi_3 55.58881996735126 54.7449085712738
conf1, conf2, E(conf1), E(conf2) =  m1_b_ss_sa_0 m1_b_sdf_2 54.05314425954641 53.65038116111539
conf1, conf2, E(conf1), E(conf2) =  m1_b_ss_sa_5 m1_b_sa_sa_8 58.49137517556325 59.11374009994782
conf1, conf2, E(conf1), E(

In [146]:
align_structures_to_lowest_energy(allmol_m1_b, energy_m1_b)
p_b = prepare_view(allmol_m1_b)
p_b.show()

In [148]:
print("The final number of conformers generated with the Balloon software after the removal of duplicates = ", len(allmol_m1_b))
print("Sorted energy of all selected conformers and the energy differences with respect to the lowest:")
energy_sorted = sorted(energy_m1_b.items(), key=lambda x: x[1])
energy_diff = []
e_min = energy_sorted[0][1]
for e in energy_sorted:
    e_diff = e[1] - e_min
    energy_diff.append([e[0], e[1], e_diff])
    
energy_diff    

The final number of conformers generated with the Balloon software after the removal of duplicates =  66
Sorted energy of all selected conformers and the energy differences with respect to the lowest:


[['m1_b_sdf_0', 53.045941690840834, 0.0],
 ['m1_b_smi_0', 53.05726584226574, 0.01132415142490828],
 ['m1_b_sdf_3', 53.71006591139638, 0.6641242205555429],
 ['m1_b_smi_2', 54.22585722309141, 1.1799155322505754],
 ['m1_b_smi_3', 54.7449085712738, 1.6989668804329625],
 ['m1_b_sdf_4', 55.067264568123825, 2.0213228772829908],
 ['m1_b_sdf_5', 55.48265722971802, 2.4367155388771877],
 ['m1_b_sa_sa_1', 55.54869755784455, 2.5027558670037138],
 ['m1_b_sdf_6', 56.22581268888335, 3.1798709980425173],
 ['m1_b_sdf_7', 56.62217643241097, 3.5762347415701328],
 ['m1_b_sa_sa_2', 56.71133416332353, 3.6653924724826936],
 ['m1_b_sa_sa_4', 56.83850060622465, 3.7925589153838146],
 ['m1_b_ss_sa_3', 56.881089421443896, 3.8351477306030617],
 ['m1_b_ss_sa_4', 57.07186921216874, 4.025927521327908],
 ['m1_b_smi_4', 57.58320727004523, 4.537265579204394],
 ['m1_b_sa_aa_0', 58.19604812689187, 5.150106436051033],
 ['m1_b_sa_aa_1', 58.24923292968118, 5.203291238840343],
 ['m1_b_smi_5', 58.43567844331593, 5.3897367524750

Estimated Boltzmann distribution of generated conformers:

## Conformers generated with the RDKit software:

In [158]:
inps_m1_rdkit_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_smiles/*.sdf')
inps_m1_rdkit_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_sdf/*.sdf')
inps_m1_rdkit_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_ss_sa/*.sdf')
inps_m1_rdkit_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_ss_aa/*.sdf')
inps_m1_rdkit_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_sa/*.sdf')
inps_m1_rdkit_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_as/*.sdf')
inps_m1_rdkit_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_aa/*.sdf')
inps_m1_rdkit_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_aa_aa/*.sdf')

In [159]:
e_m1_rdkit_smi = grep_energies_from_sdf_outputs(inps_m1_rdkit_smi)
e_m1_rdkit_sdf = grep_energies_from_sdf_outputs(inps_m1_rdkit_sdf)
e_m1_rdkit_ss_sa = grep_energies_from_sdf_outputs(inps_m1_rdkit_ss_sa)
e_m1_rdkit_ss_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_ss_aa)
e_m1_rdkit_sa_sa = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_sa)
e_m1_rdkit_sa_as = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_as)
e_m1_rdkit_sa_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_aa)
e_m1_rdkit_aa_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_aa_aa)

In [160]:
# write conformers to dictionaries
suppl_m1_rdkit_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_smiles.sdf')
suppl_m1_rdkit_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_sdf.sdf')
suppl_m1_rdkit_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_ss_sa.sdf')
suppl_m1_rdkit_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_ss_aa.sdf')
suppl_m1_rdkit_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_sa.sdf')
suppl_m1_rdkit_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_as.sdf')
suppl_m1_rdkit_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_aa.sdf')
suppl_m1_rdkit_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_aa_aa.sdf')

allmol_m1_rdkit_smi = write_to_dict("m1_rdkit_smi_", suppl_m1_rdkit_smi)
allmol_m1_rdkit_sdf = write_to_dict("m1_rdkit_sdf_", suppl_m1_rdkit_sdf)
allmol_m1_rdkit_ss_sa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_ss_sa)
allmol_m1_rdkit_ss_aa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_ss_aa)
allmol_m1_rdkit_sa_sa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_sa_sa)
allmol_m1_rdkit_sa_as = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_sa_as)
allmol_m1_rdkit_sa_aa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_sa_aa)
allmol_m1_rdkit_aa_aa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_aa_aa)

In [161]:
# align:
align_structures_to_lowest_energy(allmol_m1_rdkit_sdf, e_m1_rdkit_sdf)
align_structures_to_lowest_energy(allmol_m1_rdkit_smi, e_m1_rdkit_smi)
align_structures_to_lowest_energy(allmol_m1_rdkit_ss_sa, e_m1_rdkit_ss_sa)
align_structures_to_lowest_energy(allmol_m1_rdkit_ss_aa, e_m1_rdkit_ss_aa)
align_structures_to_lowest_energy(allmol_m1_rdkit_sa_sa, e_m1_rdkit_sa_sa)
align_structures_to_lowest_energy(allmol_m1_rdkit_sa_as, e_m1_rdkit_sa_as)
align_structures_to_lowest_energy(allmol_m1_rdkit_sa_aa, e_m1_rdkit_sa_aa)
align_structures_to_lowest_energy(allmol_m1_rdkit_aa_aa, e_m1_rdkit_aa_aa)

first =  m1_rdkit_sdf_40
moldict =  {'m1_rdkit_sdf_39': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10080>, 'm1_rdkit_sdf_13': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab12030>, 'm1_rdkit_sdf_49': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab106c0>, 'm1_rdkit_sdf_18': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10a80>, 'm1_rdkit_sdf_20': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10670>, 'm1_rdkit_sdf_16': <rdkit.Chem.rdchem.Mol object at 0x7fab08d5ff30>, 'm1_rdkit_sdf_47': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10e40>, 'm1_rdkit_sdf_9': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab124e0>, 'm1_rdkit_sdf_36': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10cb0>, 'm1_rdkit_sdf_2': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab123a0>, 'm1_rdkit_sdf_22': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10710>, 'm1_rdkit_sdf_10': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab120d0>, 'm1_rdkit_sdf_37': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab10c10>, 'm1_rdkit_sdf_7': <rdkit.Chem.rdchem.Mol object at 0x7fab0ab12120>, 

KeyError: 'm1_rdkit_ss_aa_40'

In [73]:
%%html
<table>
  <tr>
    <td id="m1_rdkit_crystal" ></td>
    <td id="m1_rdkit_smiles"  ></td>
  <tr>
    <td> m1_rdkit_crystal </td>
    <td> m1_rdkit_smiles  </td>  
  </tr>
</table>

,
m1_rdkit_crystal,m1_rdkit_smiles


In [74]:
# view:
p1_rdkit_handles=[]

p1_rdkit_sdf = py3Dmol.view(width=400,height=400)
p1_rdkit_sdf.removeAllModels()
for key, mol in allmol_m1_rdkit_sdf.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_rdkit_sdf.addModel(mb,'sdf')    
p1_rdkit_sdf.setStyle({'stick':{'radius':'0.15'}})
p1_rdkit_sdf.setBackgroundColor('0xeeeeee')
p1_rdkit_sdf.zoomTo()
p1_rdkit_handles.append(p1_rdkit_sdf)

p1_rdkit_smi = py3Dmol.view(width=400,height=400)
p1_rdkit_smi.removeAllModels()
for key, mol in allmol_m1_rdkit_smi.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_rdkit_smi.addModel(mb,'sdf')    
p1_rdkit_smi.setStyle({'stick':{'radius':'0.15'}})
p1_rdkit_smi.setBackgroundColor('0xeeeeee')
p1_rdkit_smi.zoomTo()
p1_rdkit_handles.append(p1_rdkit_smi)

In [75]:
p1_rdkit_handles[0].insert('m1_rdkit_crystal')

In [76]:
p1_rdkit_handles[1].insert('m1_rdkit_smiles')

### pre-screening

In [77]:
allmol_m1_rdkit = {}
allmol_m1_rdkit.update(allmol_m1_rdkit_sdf)
allmol_m1_rdkit.update(allmol_m1_rdkit_smi)

energy_m1_rdkit = {}
energy_m1_rdkit.update(e_m1_rdkit_sdf)
energy_m1_rdkit.update(e_m1_rdkit_smi)

rms_m1_rdkit = {}
for key, mol in allmol_m1_rdkit.items():
    rms_m1_rdkit[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m1_rdkit[key], rms_m1_rdkit[key]))

In [78]:
rms_sorted = sorted(rms_m1_rdkit.items(), key=lambda x: x[1])
rms_thresh = 0.05

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m1_rdkit[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1_rdkit, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1_rdkit[mol]
    del energy_m1_rdkit[mol]
    del rms_m1_rdkit[mol]  

List sorted by RMS:
name = m1_rdkit_smi_51, E = 53.090000, RMS = 0.830433
name = m1_rdkit_smi_12, E = 53.870000, RMS = 0.961759
name = m1_rdkit_smi_28, E = 57.820000, RMS = 1.114038
name = m1_rdkit_smi_54, E = 57.820000, RMS = 1.114052
name = m1_rdkit_sdf_23, E = 62.850000, RMS = 1.156788
name = m1_rdkit_sdf_6, E = 62.850000, RMS = 1.156812
name = m1_rdkit_smi_23, E = 57.190000, RMS = 1.208917
name = m1_rdkit_smi_48, E = 58.960000, RMS = 1.218295
name = m1_rdkit_sdf_30, E = 60.250000, RMS = 1.256433
name = m1_rdkit_smi_24, E = 61.810000, RMS = 1.269369
name = m1_rdkit_sdf_54, E = 56.020000, RMS = 1.276148
name = m1_rdkit_smi_13, E = 60.350000, RMS = 1.280484
name = m1_rdkit_sdf_45, E = 57.260000, RMS = 1.280507
name = m1_rdkit_sdf_32, E = 62.220000, RMS = 1.282954
name = m1_rdkit_sdf_43, E = 62.220000, RMS = 1.283010
name = m1_rdkit_sdf_11, E = 55.420000, RMS = 1.298893
name = m1_rdkit_sdf_8, E = 61.250000, RMS = 1.302636
name = m1_rdkit_smi_27, E = 60.350000, RMS = 1.315356
name = m1_

Below we will align the selected conformers:

In [79]:
for key, mol in allmol_m1_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p_r = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1_rdkit.items():
    mb = Chem.MolToMolBlock(mol)
    p_r.addModel(mb,'sdf')
p_r.setStyle({'stick':{'radius':'0.15'}})
p_r.setBackgroundColor('0xeeeeee')
p_r.zoomTo()
p_r.show()

## Summary

Now let's generate a list of all conformers (from all programs used, as presented above). We can further pre-screen all the structures and remove potential duplicates. Here we can also use more crude RMSD threshold.

In [80]:
allmol_m1 = {}
allmol_m1.update(allmol_m1_b)
allmol_m1.update(allmol_m1_rdkit)

energy_m1 = {}
energy_m1.update(energy_m1_b)
energy_m1.update(energy_m1_rdkit)

rms_m1 = {}
for key, mol in allmol_m1.items():
    rms_m1[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m1[key], rms_m1[key]))

Now we will calculate the similarity matrix, which shows the RMSD value for any two conformers. 

We will print the similarity matrix for pairs of conformers from the lowest value of the RMSD (= the most similar structures) to the highest values of the RMSD (= the least similar structures):

In [131]:
similarity_matrix = {}

for k1, m1 in allmol_m1.items():
    for k2, m2 in allmol_m1.items():
        if (k1, k2) in similarity_matrix.keys() or (k2, k1) in similarity_matrix.keys():
            pass
        else:
            if k1 != k2:
                rms = AllChem.GetBestRMS(Chem.RemoveHs(m1),Chem.RemoveHs(m2))
                similarity_matrix[(k1, k2)] = rms 
    
similarity_matrix_sorted = sorted(similarity_matrix.items(), key=lambda x: x[1])

 
similarity_matrix_sorted

[(('m1_b_sdf_3', 'm1_b_smi_1'), 0.24115128238421304),
 (('m1_b_smi_3', 'm1_b_sdf_7'), 0.25656055772360714),
 (('m1_rdkit_sdf_4', 'm1_rdkit_smi_32'), 0.2769131707043658),
 (('m1_b_smi_1', 'm1_b_sdf_2'), 0.2934010528718048),
 (('m1_b_sdf_1', 'm1_b_smi_0'), 0.31648967158944663),
 (('m1_b_sdf_1', 'm1_b_smi_3'), 0.4038783982705559),
 (('m1_b_sdf_2', 'm1_b_smi_0'), 0.43996892965854084),
 (('m1_b_smi_1', 'm1_b_sdf_4'), 0.49684477972891106),
 (('m1_b_sdf_3', 'm1_b_sdf_2'), 0.5005633158792928),
 (('m1_b_sdf_3', 'm1_b_sdf_4'), 0.5133161326979814),
 (('m1_b_sdf_0', 'm1_b_sdf_2'), 0.5249532740261654),
 (('m1_b_smi_3', 'm1_b_smi_0'), 0.5272276991704827),
 (('m1_b_sdf_0', 'm1_b_smi_3'), 0.5384632241251002),
 (('m1_b_sdf_1', 'm1_b_sdf_7'), 0.547244539084709),
 (('m1_b_smi_3', 'm1_b_sdf_2'), 0.5496029195099067),
 (('m1_rdkit_sdf_4', 'm1_rdkit_sdf_9'), 0.5584669709845531),
 (('m1_b_sdf_0', 'm1_b_sdf_7'), 0.5969718529418608),
 (('m1_b_sdf_0', 'm1_b_smi_0'), 0.6079091648830538),
 (('m1_b_smi_1', 'm1_b_sm

In [ ]:
rms_sorted = sorted(rms_m1.items(), key=lambda x: x[1])
rms_thresh = 0.1

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1[mol]
    del energy_m1[mol]
    del rms_m1[mol]  

Finally we can align all conformers which will further be used as starting points in DFT geometry optimizations:

In [ ]:
print(len(allmol_m1))

for key, mol in allmol_m1.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1.items():
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

Write the selected conformers' names to the list "list_selected_conformers_from_balloon_rdkit". It will be used to generate Gaussian inputs:

In [ ]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1/list_selected_conformers_from_ballon_rdkit", "w") as f:
    for key, mol in allmol_m1.items():
        f.write(key+"\n")

energy_sorted = sorted(energy_m1.items(), key=lambda x: x[1])
print(energy_sorted)
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1/detailed_list_selected_conformers_from_ballon_rdkit", "w") as f:
    for pair in energy_sorted:
        f.write("{0:30}   {1}\n".format(pair[0], pair[1]))        